In [6]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

train_data = pd.read_csv('X_train.csv', index_col=0)
train_labels = pd.read_csv('y_train.csv', index_col=0)
test_data = pd.read_csv('X_test.csv', index_col=0)
additionnal_data = pd.read_csv('supplementary_data_Vkoyn8z.csv', index_col=0)

In [7]:
# Additionnal data 
CLASS_1 = [] 
CLASS_2 = []
CLASS_3 = []
CLASS_4 = []
for element in tqdm(train_data["ID_TARGET"].values):
    CLASS_1.append(additionnal_data.loc[element]["CLASS_LEVEL_1"])
    CLASS_2.append(additionnal_data.loc[element]["CLASS_LEVEL_2"])
    CLASS_3.append(additionnal_data.loc[element]["CLASS_LEVEL_3"])
    CLASS_4.append(additionnal_data.loc[element]["CLASS_LEVEL_4"])

  0%|          | 0/267100 [00:00<?, ?it/s]

In [8]:
train_data["CLASS_LEVEL_1"] = CLASS_1
train_data["CLASS_LEVEL_2"] = CLASS_2
train_data["CLASS_LEVEL_3"] = CLASS_3
train_data["CLASS_LEVEL_4"] = CLASS_4
train_data["RET_TARGET"] = train_labels.values

In [9]:
train_data.head()

,ID_DAY,RET_216,RET_238,RET_45,RET_295,RET_230,RET_120,RET_188,RET_260,RET_15,...,RET_193,RET_95,RET_162,RET_297,ID_TARGET,CLASS_LEVEL_1,CLASS_LEVEL_2,CLASS_LEVEL_3,CLASS_LEVEL_4,RET_TARGET
ID,,,,,,,,,,,,,,,,,,,,,
0,3316,0.004024,0.009237,0.004967,NaN,0.01704,0.013885,0.041885,0.015207,-0.003143,...,-0.010233,0.001251,-0.003102,-0.094847,139,0,0,1,1,-0.022351
1,3316,0.004024,0.009237,0.004967,NaN,0.01704,0.013885,0.041885,0.015207,-0.003143,...,-0.010233,0.001251,-0.003102,-0.094847,129,6,13,33,53,-0.011892
2,3316,0.004024,0.009237,0.004967,NaN,0.01704,0.013885,0.041885,0.015207,-0.003143,...,-0.010233,0.001251,-0.003102,-0.094847,136,6,13,33,53,-0.015285
3,3316,0.004024,0.009237,0.004967,NaN,0.01704,0.013885,0.041885,0.015207,-0.003143,...,-0.010233,0.001251,-0.003102,-0.094847,161,3,5,20,32,-0.019226
4,3316,0.004024,0.009237,0.004967,NaN,0.01704,0.013885,0.041885,0.015207,-0.003143,...,-0.010233,0.001251,-0.003102,-0.094847,217,4,10,28,47,0.006644


In [10]:
# Replace NaN with mean
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
train_data = imp.fit_transform(train_data)

In [11]:
columns = list(test_data.columns) + ["CLASS_LEVEL_1", "CLASS_LEVEL_2", "CLASS_LEVEL_3", "CLASS_LEVEL_4", "RET_TARGET"]
train_data = pd.DataFrame(train_data, columns=columns)

In [12]:
# Scaling data with Robust Scaler model
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(train_data.drop("RET_TARGET", axis=1))
scaled_X = scaler.transform(train_data.drop("RET_TARGET", axis=1))

In [13]:
# Run a PCA to have more reliable data
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

model = PCA()
coord = model.fit_transform(scaled_X)

plt.plot(np.arange(1, coord.shape[1]+1), np.cumsum(model.explained_variance_ratio_))
plt.grid()
plt.title("Explained variance ratio vs. # of components")
plt.xlabel("# of components")
plt.ylabel("% of variance explained")

Text(0,0.5,'% of variance explained')

In [14]:
# Feature selection
from sklearn.feature_selection import SelectKBest, f_classif
coord_new = SelectKBest(f_classif, k=60).fit_transform(coord, train_data["RET_TARGET"].values)

In [15]:
# Split the DataSet in train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(coord_new, train_data["RET_TARGET"], 
                                                    test_size=0.2, random_state=42)

In [ ]:
# Train a classifier to predict the asset return sign
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

param_grid = {"learning_rate": [.01, .02, .03],
              "n_estimators": [100, 200, 300],
              "max_depth": [3, 4, 5]}

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=5, n_jobs=-1, verbose=1)
grid.fit(X_train, np.sign(y_train))

print("Lowest error", grid.best_score_)
print()

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 94.2min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 334.3min finished


In [23]:
from sklearn.ensemble import GradientBoostingClassifier

# Choosing best model
model = GradientBoostingClassifier(learning_rate=0.01, n_estimators=100, 
                                   max_depth=3, verbose=1)
model.fit(X_train, np.sign(y_train))

# Predictions on test set
y_pred = model.predict(X_test)

# Score metric used for the challenge
def weighted_accuracy(y_test, y_pred):
    y_abs = np.abs(y_test)
    norm = y_abs.sum()
    score = ((np.sign(y_pred) == np.sign(y_test)) * y_abs).sum() / norm
    return score

from sklearn.metrics import confusion_matrix

print("Confusion matrix :")
print(confusion_matrix(y_true=np.sign(y_test), y_pred=model.predict(X_test)))
print()
print("Score :")
print(weighted_accuracy(y_test, model.predict(X_test)))

      Iter       Train Loss   Remaining Time 
         1      147721.9548           18.87m
         2      147509.0723           18.72m
         3      147301.8103           18.53m
         4      147099.9916           18.14m
         5      146903.4344           18.01m
         6      146711.9901           17.99m
         7      146524.8245           17.82m
         8      146343.1391           17.62m
         9      146165.5036           17.38m
        10      145993.0513           17.09m
        20      144489.7792           15.07m
        30      143324.5225           13.15m
        40      142418.8901           11.23m
        50      141707.3474            9.35m
        60      141150.0701            7.48m
        70      140711.2565            5.60m
        80      140361.9526            3.73m
        90      140079.0616            1.87m
       100      139849.6763            0.00s
Confusion matrix :
[[19462     0  8583]
 [    1     0     0]
 [11884     0 13490]]

Score :
0.67821